In [72]:
def charConvert():
    file1 = open('wiki_00', 'r')

    file2 = open('converted.txt', 'w')

    for line in file1.readlines():

        if not (line.startswith('<')):
            line = line.replace("İ","I")
            line = line.replace("Ö","O")
            line = line.replace("Ü","U")
            line = line.replace("Ş","S")
            line = line.replace("Ç","C")
            line = line.replace("Ğ","G")

            line = line.replace("ı","i")
            line = line.replace("ö","o")
            line = line.replace("ü","u")
            line = line.replace("ş","s")
            line = line.replace("ç","c")
            line = line.replace("ğ","g")
            
            line = line.lower()

            file2.write(line)

    file2.close()
    file1.close()


In [73]:
# %pip install git+https://github.com/ftkurt/python-syllable.git@master

from syllable import Encoder

def syllableParse():
    file1 = open('converted.txt', 'r')
    file2 = open('syllables.txt', 'w')


    encoder = Encoder(lang="tr", limitby="vocabulary", limit=3000)



    for line in file1.readlines():
        lineAr = line.split()
        for word in lineAr:
            file2.write(encoder.tokenize(word))
            if word.endswith('.'):
                file2.write(' nkt ')
            file2.write(' spc ')
        file2.write('\n')

    file1.close()
    file2.close()

In [74]:
numSyllables = 0

NUMSYLLABLES = 189022929
NUMSYLLABLES95 = 180439320
NUMUNIQUESYL = 2218

In [75]:
def insertionSort(arr, sylArr):
    for i in range(1, len(arr)):
        key = arr[i]
        sylKey = sylArr[i]
        j = i-1
        while j >= 0 and int(key) < int(arr[j]):
            sylArr[j+1] = sylArr[j]
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
        sylArr[j+1] = sylKey

In [76]:
def percent():
    file1 = open('syllables.txt', 'r')
    file2 = open('syllables95.txt', 'w')
    file3 = open('syllables5.txt', 'w')


    3920807
    3724766.65
    for i in range (3724766):
        line = file1.readline()
        file2.write(line)

    for i in range (196041):
        line = file1.readline()
        file3.write(line)

    file1.close()
    file2.close()
    file3.close()


In [77]:
def create_unigram():
    file1 = open('syllables95.txt', 'r')
    file2 = open('unigram95.txt', 'w')
    global numSyllables
    sylArr = []
    sylCounts = []
    for line in file1.readlines():
        sylls = line.split()
        numSyllables += len(sylls)
        for syl in sylls:
            try:
                ind = sylArr.index(syl)
                sylCounts[ind] += 1
            except:
                sylArr.append(syl)
                sylCounts.append(1)

    print("Number of syllables: " + str(numSyllables))
    print("Number of unique syllables: " + str(len(sylArr)) + " " + str(len(sylCounts)))

    insertionSort(sylCounts, sylArr)

    for i in range (len(sylArr)):
        file2.write(sylArr[i] + ' ' + str(sylCounts[i]) + '\n')

    file1.close()
    file2.close()

In [78]:
sylArr = []
sylCounts = []

def read_unigram():
    file1 = open('unigram95.txt', 'r')

    for line in file1.readlines():
        sylls = line.split()
        sylArr.append(sylls[0])
        sylCounts.append(sylls[1])

    file1.close()

In [79]:
indexDict = {}

def arrtoDict():
    for i in range(len(sylArr)):
        indexDict[sylArr[i]] = i

In [80]:
class NDSparseMatrix:
  def __init__(self):
    self.elements = {}

  def addValue(self, tuple, value):
    self.elements[tuple] = value

  def readValue(self, tuple):
    try:
      value = self.elements[tuple]
    except KeyError:
      # could also be 0.0 if using floats...
      value = 0
    return value

In [81]:
bigramArr = NDSparseMatrix()

def create_bigram():
    file1 = open('syllables95.txt', 'r')

    print("Create Bigram")

    for line in file1.readlines():
        sylls = line.split()
        for i in range (1, len(sylls)):
            bigramArr.addValue((indexDict[sylls[i-1]], indexDict[sylls[i]]), bigramArr.readValue((indexDict[sylls[i-1]], indexDict[sylls[i]])) + 1)

    print("Bigram is created.")

    file1.close()

In [82]:

trigramArr = NDSparseMatrix()


def create_trigram():
    file1 = open('syllables95.txt', 'r')

    print("Create trigram")

    for line in file1.readlines():
        sylls = line.split()
        for i in range (2, len(sylls)):
            trigramArr.addValue((indexDict[sylls[i-2]], indexDict[sylls[i-1]], indexDict[sylls[i]]), trigramArr.readValue((indexDict[sylls[i-2]], indexDict[sylls[i-1]], indexDict[sylls[i]])) + 1) 

    print("Trigram is created.")

    file1.close()

In [83]:
gt_prob_bi = 0
gt_prob_uni = 0
gt_prob_tri = 0
def GTSmooth():
    global gt_prob_bi
    global gt_prob_uni
    global gt_prob_tri

    print("GT Smoothing bigram.")
    for i in range (NUMUNIQUESYL):
        for j in range (NUMUNIQUESYL):
            if(bigramArr.readValue((i, j)) == 1):
              gt_prob_bi += 1

    print("GT Smoothing unigram.")
    for i in range (len(sylArr)):
        if(sylCounts[i] == 1):
            gt_prob_uni += 1
    print("GT Smoothing trigram.")
    # for i in range (NUMUNIQUESYL):
    #     for j in range (NUMUNIQUESYL):
    #         for k in range (NUMUNIQUESYL):
    #             if(trigramArr.readValue((i, j, k)) == 1):
    #                 gt_prob_tri +=1
    print("GT Smoothing finished.")

In [92]:
def perplexity_unigram(sent):
    senPerp = 1
    for i in range(0, len(sent)):
        val = 0
        if(sylCounts[indexDict[sent[i]]] == 0):
            val = gt_prob_uni
        else:
            val = sylCounts[indexDict[sent[i]]]
        senPerp = senPerp * ((NUMSYLLABLES + gt_prob_uni)/(int(val)))

    senPerp = senPerp**(1/len(sent))
    
    return senPerp

In [93]:
def perplexity_bigram(sent):
    senPerp = 1
    for i in range(1, len(sent)):
        tot = 0
        for j in range (NUMUNIQUESYL):
            tot += bigramArr.readValue((indexDict[sent[i-1]], indexDict[sylArr[j]]))
        val = 0
           
        if(bigramArr.readValue((indexDict[sent[i-1]], indexDict[sent[i]])) == 0):
            val = gt_prob_bi
        else:
            val = bigramArr.readValue((indexDict[sent[i-1]], indexDict[sent[i]]))
        
        senPerp = senPerp * ((tot+gt_prob_bi)/(val))
        
    senPerp = senPerp**(1/len(sent))
    
    return senPerp

    

In [94]:
def perplexity_trigram(sent):
    senPerp = 1
    for i in range(2, len(sent)):
        tot = 0
        for j in range (NUMUNIQUESYL):
            tot += trigramArr.readValue((indexDict[sent[i-2]], indexDict[sent[i-1]], indexDict[sylArr[j]]))
        
        val = 0
        if(trigramArr.readValue((indexDict[sent[i-2]], indexDict[sent[i-1]], indexDict[sent[i]])) == 0):
            val = 1
        else:
            val = trigramArr.readValue((indexDict[sent[i-2]], indexDict[sent[i-1]], indexDict[sent[i]])) + 1
        
        senPerp = senPerp * (tot/(val))
            
    senPerp = senPerp**(1/len(sent))
    
    return senPerp

In [154]:
def perplexity_test():
    file1 = open('syllables5.txt', 'r')
    file2 = open('perplexity.txt', 'w')

    sylls = file1.read()
    sylls = sylls.split()

    sentenceArray = []
    queue = []

    for syl in sylls:
        queue.append(syl)
        if(syl == "nkt"):
            sentenceArray.append(queue)
            queue = []

    print("Perplexity calculation started.")
    count = 0
    for sent in sentenceArray:
        if(count == 1000):
            print("1000 sentences are calculated.")
            break
        count += 1
        str = ""
        for syl in sent:
            if(syl == "spc"):
                str += " "
            elif(syl == "nkt"):
                str += "."
            else:
                str += syl
        print("Sentence: " + str, file=file2)
        print("Perplexity Unigram:  ", file=file2, end="")
        print(perplexity_unigram(sent), file=file2)
        print("Perplexity Bigram:   ", file=file2, end="")
        print(perplexity_bigram(sent), file=file2)
        print("Perplexity Trigram:  ", file=file2, end="")
        print(perplexity_trigram(sent), file=file2)
        print("\n", file=file2)

    file1.close()
    file2.close()
        


In [155]:
import random

def generateSentenceUnigram():
    sent = []
    sylQueue = []

    i = len(sylArr) - 3
    
    for j in range (6):
        sylQueue.append(sylArr[i])
        i -= 1
    
    pos = random.randint(0,5)

    sent.append(sylQueue[pos])
    sylQueue.remove(sylQueue[pos])

    endpoint = 5
    while True:
        if(sent.count("spc") >= 4):
            endpoint = 6

        pos = random.randint(0,endpoint)

        if(pos == 5):
            if(sent[len(sent) - 1] != "spc"):
                sent.append("spc")

        elif(pos == 6):
            if(sent[len(sent) - 1] != "spc"):
                sent.append("nkt")
                break

        else:
            sent.append(sylQueue[pos])
            sylQueue.remove(sylQueue[pos])
            sylQueue.append(sylArr[i])
            i -= 1

    str = ""
    for syl in sent:
        if(syl == "spc"):
            str += " "
        elif(syl == "nkt"):
            str += "."
        else:
            str += syl
    return str
        

In [156]:
def generateSentenceBigram():
    sent = []
    sylQueue = []
    pos = random.randint(0,NUMUNIQUESYL - 1)

    sent.append(sylArr[pos])

    endpoint = 5
    while True:
        lastSyl = sent[len(sent) - 1]

        # if(lastSyl == "spc"):
        #     pos = random.randint(0,NUMUNIQUESYL - 1)
        #     sent.append(sylArr[pos])
        #     continue

        sylQueue = []
        for j in range (5):
            pos = random.randint(0,NUMUNIQUESYL - 1)
            sylQueue.append(sylArr[pos])

        

        for k in range (NUMUNIQUESYL):
            
            min = bigramArr.readValue((indexDict[lastSyl], indexDict[sylQueue[0]]))
            minSyl = sylQueue[0]
            for j in range (1, 5):
                if (bigramArr.readValue((indexDict[lastSyl], indexDict[sylQueue[j]])) < min):
                    min = bigramArr.readValue((indexDict[lastSyl], indexDict[sylQueue[j]]))
                    minSyl = sylQueue[j]
            
            for j in range (5):
                if(bigramArr.readValue((indexDict[lastSyl], indexDict[sylArr[k]])) > bigramArr.readValue((indexDict[lastSyl], indexDict[sylQueue[j]]))):
                    sylQueue.remove(minSyl)
                    sylQueue.append(sylArr[k])
                    break
        
        
        if(sent.count("spc") >= 4):
            endpoint = 6

        pos = random.randint(0,endpoint)

        if(pos == 5):
            if(sent[len(sent) - 1] != "spc"):
                sent.append("spc")

        elif(pos == 6):
            if(sent[len(sent) - 1] != "spc"):
                sent.append("nkt")
                break

        else:
            sent.append(sylQueue[pos])
            sylQueue.remove(sylQueue[pos])
            #sylQueue.append(sylArr[i])
            #i -= 1
        
    
    str = ""
    for syl in sent:
        if(syl == "spc"):
            str += " "
        elif(syl == "nkt"):
            str += "."
        else:
            str += syl
    return str



    

In [157]:
def generateSentenceTrigram():
    sent = []
    sylQueue = []
    pos = random.randint(0, NUMUNIQUESYL - 1)

    sent.append(sylArr[pos])

    maxBi = bigramArr.readValue((indexDict[sent[0]], indexDict[sylArr[0]]))
    ind = 0
    for j in range (NUMUNIQUESYL):
        if (bigramArr.readValue((indexDict[sent[0]], indexDict[sylArr[j]])) > maxBi):
            maxBi = bigramArr.readValue((indexDict[sent[0]], indexDict[sylArr[j]]))
            ind = j
    sent.append(sylArr[ind])

    
    endpoint = 5
    while True:
        lastSyl = sent[len(sent) - 1]
        prevSyl = sent[len(sent) - 2]

        sylQueue = []

        for j in range (5):
            pos = random.randint(0,NUMUNIQUESYL - 1)
            sylQueue.append(sylArr[pos])

        
        for k in range (NUMUNIQUESYL):
            
            min = trigramArr.readValue((indexDict[prevSyl], indexDict[lastSyl], indexDict[sylQueue[0]]))
            minSyl = sylQueue[0]
            for j in range (1, 5):
                if (trigramArr.readValue((indexDict[prevSyl], indexDict[lastSyl], indexDict[sylQueue[j]])) < min):
                    min = trigramArr.readValue((indexDict[prevSyl], indexDict[lastSyl], indexDict[sylQueue[j]]))
                    minSyl = sylQueue[j]
            
            for j in range (5):
                if(trigramArr.readValue((indexDict[prevSyl], indexDict[lastSyl], indexDict[sylArr[k]])) > trigramArr.readValue((indexDict[prevSyl], indexDict[lastSyl], indexDict[sylQueue[j]]))):
                    sylQueue.remove(minSyl)
                    sylQueue.append(sylArr[k])
                    break

        if(sent.count("spc") >= 4):
            endpoint = 6

        pos = random.randint(0,endpoint)

        if(pos == 5):
            if(sent[len(sent) - 1] != "spc"):
                sent.append("spc")

        elif(pos == 6):
            if(sent[len(sent) - 1] != "spc"):
                sent.append("nkt")
                break

        else:
            sent.append(sylQueue[pos])
            sylQueue.remove(sylQueue[pos])
            #sylQueue.append(sylArr[i])
            #i -= 1
        
    
    str = ""
    for syl in sent:
        if(syl == "spc"):
            str += " "
        elif(syl == "nkt"):
            str += "."
        else:
            str += syl
    return str

In [ ]:
charConvert()
syllableParse()
percent()
create_unigram()
read_unigram()
arrtoDict()
create_bigram()
create_trigram()
GTSmooth()

perplexity_test()


file1 = open("unigramSentences.txt", 'w')
file2 = open("bigramSentences.txt", "w")
file3 = open("trigramSentences.txt", "w")

for i in range (100):
    print(generateSentenceUnigram(), file=file1)
    print(generateSentenceBigram(), file=file2)
    print(generateSentenceTrigram(), file=file3)

file1.close()
file2.close()
file3.close()




